In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import io

In [2]:
positions = ["Abwehr", "Mittelfeld", "Sturm"]
pages = 20

In [4]:
def get_players_from_page(position, page):
    url = f"https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop/plus/0/galerie/0?ausrichtung={position}&spielerposition_id=alle&altersklasse=alle&jahrgang=0&land_id=0&kontinent_id=0&yt0=Show&page={page}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    players = []
    table = soup.find('table', {'class': 'items'})
    if table:
        for row in table.find_all('tr')[1:]:  
            cols = row.find_all('td')
            if len(cols) >= 8:
                name_cell = cols[1].find('table').find_all('tr')[0].find_all('td')[1].find('a')
                if name_cell:
                    player = {
                        'Name': name_cell.text.strip(),
                        'Market Value': cols[8].text.strip()
                    }
                    players.append(player)
    return players

In [34]:
all_players = []

In [35]:
for position in positions:
    for page in range(1, pages + 1):
        players = get_players_from_page(position, page)
        all_players.extend(players)

In [36]:
df = pd.DataFrame(all_players)

In [37]:
df

,Name,Market Value
0,William Saliba,€80.00m
1,Rúben Dias,€80.00m
2,Josko Gvardiol,€75.00m
3,Ronald Araujo,€70.00m
4,Gabriel Magalhães,€70.00m
...,...,...
1495,Mika Biereth,€7.00m
1496,Osman Bukari,€7.00m
1497,Michael,€7.00m
1498,Liel Abada,€7.00m


In [22]:
df = df.rename(columns={'Name':'Player'})

In [23]:
df.to_csv('Market Value.csv')

In [7]:
df = pd.read_csv('Market Value.csv')

In [4]:
df_f = pd.read_csv('Final.csv')

In [5]:
df_f

,Unnamed: 0,Rk,Player,Nation,Pos,Squad,Comp,Age,MP,Starts,...,TklW90,TklADribblers90,TklAttDri90,Tkl%,Bl90,ShBl90,PassBl90,Int90,Clr90,Err90
0,0,1,Max Aarons,ENG,DF,Bournemouth,Premier League,23.0,20,13,...,1.39,1.46,2.48,58.8,0.66,0.36,0.29,0.58,1.97,0.00
1,1,2,Brenden Aaronson,USA,"MF,FW",Union Berlin,Bundesliga,22.0,30,14,...,1.28,1.13,2.27,50.0,1.84,0.07,1.77,0.14,0.28,0.00
2,4,5,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,35.0,31,31,...,1.13,0.84,1.46,57.8,1.65,1.04,0.61,1.26,3.53,0.06
3,5,6,Salis Abdul Samed,GHA,MF,Lens,Ligue 1,23.0,27,17,...,0.83,0.47,1.07,44.4,0.71,0.06,0.65,0.71,1.07,0.00
4,7,8,Laurent Abergel,FRA,MF,Lorient,Ligue 1,30.0,33,32,...,1.64,1.19,3.02,39.6,0.91,0.19,0.72,1.92,1.92,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,2845,2846,Martín Zubimendi,ESP,MF,Real Sociedad,La Liga,24.0,31,29,...,0.92,0.78,1.42,54.8,1.12,0.31,0.81,1.25,1.83,0.00
1690,2846,2847,Szymon Żurkowski,POL,MF,Empoli,Serie A,25.0,13,10,...,0.62,0.52,1.56,33.3,0.52,0.10,0.42,0.62,1.15,0.00
1691,2848,2849,Martin Ødegaard,NOR,MF,Arsenal,Premier League,24.0,35,35,...,0.50,0.50,1.69,29.3,0.67,0.03,0.64,0.44,0.15,0.00
1692,2849,2850,Milan Đurić,BIH,FW,Hellas Verona,Serie A,33.0,20,13,...,0.00,0.00,0.22,0.0,0.00,0.00,0.00,0.07,0.97,0.00


In [8]:
final = pd.merge(df, df_f, on="Player")

In [9]:
final

,Unnamed: 0_x,Player,Market Value,Unnamed: 0_y,Rk,Nation,Pos,Squad,Comp,Age,...,TklW90,TklADribblers90,TklAttDri90,Tkl%,Bl90,ShBl90,PassBl90,Int90,Clr90,Err90
0,0,William Saliba,€80.00m,2317,2318,FRA,DF,Arsenal,Premier League,22.0,...,0.68,0.53,0.71,74.1,0.84,0.45,0.39,0.76,2.13,0.08
1,1,Rúben Dias,€80.00m,702,703,POR,DF,Manchester City,Premier League,26.0,...,0.70,0.67,0.92,73.1,1.13,0.60,0.53,1.06,2.71,0.00
2,4,Gabriel Magalhães,€70.00m,1554,1555,BRA,DF,Arsenal,Premier League,25.0,...,0.83,0.50,0.65,77.3,1.30,0.98,0.33,0.92,2.96,0.03
3,5,Alessandro Bastoni,€70.00m,233,234,ITA,DF,Inter,Serie A,24.0,...,1.31,1.03,1.59,65.0,0.99,0.44,0.56,0.91,1.90,0.04
4,6,Trent Alexander-Arnold,€70.00m,70,71,ENG,DF,Liverpool,Premier League,24.0,...,1.09,1.13,3.64,31.0,0.92,0.08,0.84,1.30,2.01,0.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,1462,Thomas Müller,€8.00m,1828,1829,GER,"MF,FW",Bayern Munich,Bundesliga,33.0,...,0.87,0.71,1.90,37.1,1.20,0.00,1.20,0.16,0.33,0.00
808,1468,Zito Luvumbo,€7.50m,1541,1542,ANG,"FW,MF",Cagliari,Serie A,21.0,...,0.28,0.19,0.69,26.7,0.88,0.00,0.88,0.05,0.09,0.00
809,1474,Christian Kouamé,€7.50m,1369,1370,CIV,FW,Fiorentina,Serie A,25.0,...,0.33,0.50,0.83,60.0,1.32,0.00,1.32,0.17,0.83,0.00
810,1481,Luca Koleosho,€7.00m,1353,1354,ITA,"MF,FW",Burnley,Premier League,18.0,...,0.37,0.19,0.93,20.0,1.20,0.00,1.20,0.19,0.00,0.00


In [10]:
final = final.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'Rk'], axis=1)

In [11]:
final['Market Value'] = final['Market Value'].str.replace('€', '').str.replace('m', '').astype(float)

In [13]:
final = final.sort_values(by="Market Value", ascending=False)

In [14]:
final

,Player,Market Value,Nation,Pos,Squad,Comp,Age,MP,Starts,Min,...,TklW90,TklADribblers90,TklAttDri90,Tkl%,Bl90,ShBl90,PassBl90,Int90,Clr90,Err90
307,Jude Bellingham,180.0,ENG,MF,Real Madrid,La Liga,20.0,28,27,2315,...,1.32,0.47,1.40,33.3,2.14,0.19,1.95,0.82,0.66,0.00
580,Kylian Mbappé,180.0,FRA,FW,Paris S-G,Ligue 1,24.0,29,24,2158,...,0.04,0.17,0.38,44.4,0.17,0.00,0.17,0.08,0.08,0.00
579,Erling Haaland,180.0,NOR,FW,Manchester City,Premier League,23.0,31,29,2552,...,0.11,0.07,0.11,66.7,0.39,0.04,0.35,0.07,0.56,0.00
581,Phil Foden,150.0,ENG,"FW,MF",Manchester City,Premier League,23.0,35,33,2857,...,0.63,0.41,0.98,41.9,0.95,0.00,0.95,0.41,0.57,0.03
582,Bukayo Saka,140.0,ENG,FW,Arsenal,Premier League,21.0,35,35,2919,...,1.08,0.80,1.82,44.1,1.33,0.06,1.27,0.46,0.74,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,Hugo Bueno,6.0,ESP,"DF,MF",Wolves,Premier League,20.0,22,7,741,...,1.71,1.46,2.32,63.2,2.07,0.49,1.59,0.73,2.93,0.00
295,Yerson Mosquera,6.0,COL,DF,Villarreal,La Liga,22.0,16,14,1229,...,0.95,0.88,1.09,80.0,0.88,0.15,0.73,1.24,4.74,0.00
294,Bamo Meïté,6.0,CIV,DF,Marseille,Ligue 1,21.0,16,6,638,...,1.41,1.13,1.69,66.7,1.55,0.70,0.85,1.55,4.93,0.00
300,Josh Doig,6.0,SCO,DF,Hellas Verona,Serie A,21.0,12,11,830,...,0.98,1.41,1.85,76.5,1.30,0.33,0.98,0.43,1.85,0.11


In [25]:
final.to_csv('Ready.csv')